In [ ]:
!pip install streamlit pyngrok nest-asyncio pandas networkx matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 23.1 MB/s eta 0:00:00


In [ ]:
%%bash
cat > /content/optimal_bst.py <<'PY'

# optimal_bst.py
from dataclasses import dataclass
from typing import List, Optional, Dict, Any, Tuple
import math
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

@dataclass
class Node:
    key: str
    index: int
    left: Optional['Node'] = None
    right: Optional['Node'] = None

    def to_dict(self) -> Dict[str, Any]:
        return {
            'key': self.key,
            'index': self.index,
            'left': self.left.to_dict() if self.left else None,
            'right': self.right.to_dict() if self.right else None,
        }

def _validate_inputs(terms: List[str], p: List[float], q: List[float]) -> None:
    n = len(terms)
    if len(p) != n:
        raise ValueError(f"len(p) debe ser {n}, pero es {len(p)}")
    if len(q) != n + 1:
        raise ValueError(f"len(q) debe ser {n+1}, pero es {len(q)}")
    for idx, val in enumerate(p):
        if val < 0: raise ValueError(f"p[{idx}] es negativo: {val}")
    for idx, val in enumerate(q):
        if val < 0: raise ValueError(f"q[{idx}] es negativo: {val}")

def _optimal_bst_clrs(terms: List[str], p: List[float], q: List[float]) -> Tuple[float, List[List[int]], List[List[float]], List[List[float]]]:
    n = len(terms)
    e = [[0.0]*(n+2) for _ in range(n+2)]
    w = [[0.0]*(n+2) for _ in range(n+2)]
    root = [[0]*(n+1) for _ in range(n+1)]
    for i in range(1, n+2):
        e[i][i-1] = q[i-1]
        w[i][i-1] = q[i-1]
    for length in range(1, n+1):
        for i in range(1, n-length+2):
            j = i + length - 1
            e[i][j] = math.inf
            w[i][j] = w[i][j-1] + p[j-1] + q[j]
            for r in range(i, j+1):
                t = e[i][r-1] + e[r+1][j] + w[i][j]
                if t < e[i][j]:
                    e[i][j] = t
                    root[i][j] = r
    return e[1][n], root, e, w

def _optimal_bst_knuth(terms: List[str], p: List[float], q: List[float]) -> Tuple[float, List[List[int]], List[List[float]], List[List[float]]]:
    n = len(terms)
    e = [[0.0]*(n+2) for _ in range(n+2)]
    w = [[0.0]*(n+2) for _ in range(n+2)]
    root = [[0]*(n+1) for _ in range(n+1)]
    for i in range(1, n+2):
        e[i][i-1] = q[i-1]
        w[i][i-1] = q[i-1]
    for i in range(1, n+1):
        j = i
        w[i][j] = w[i][j-1] + p[j-1] + q[j]
        e[i][j] = e[i][j-1] + e[j+1][j] + w[i][j]
        root[i][j] = i
    for length in range(2, n+1):
        for i in range(1, n-length+2):
            j = i + length - 1
            w[i][j] = w[i][j-1] + p[j-1] + q[j]
            r_start = root[i][j-1] if root[i][j-1] != 0 else i
            r_end = root[i+1][j] if root[i+1][j] != 0 else j
            r_start = max(r_start, i)
            r_end = min(r_end, j)
            e[i][j] = math.inf
            best_r = r_start
            for r in range(r_start, r_end+1):
                t = e[i][r-1] + e[r+1][j] + w[i][j]
                if t < e[i][j]:
                    e[i][j] = t
                    best_r = r
            root[i][j] = best_r
    return e[1][n], root, e, w

def optimal_bst(terms: List[str], p: List[float], q: List[float], method: str = 'knuth') -> Dict[str, Any]:
    _validate_inputs(terms, p, q)
    method = method.lower().strip()
    if method == 'clrs':
        cost, root, e, w = _optimal_bst_clrs(terms, p, q)
    elif method == 'knuth':
        cost, root, e, w = _optimal_bst_knuth(terms, p, q)
    else:
        raise ValueError("method debe ser 'clrs' o 'knuth'.")
    return {'cost': cost, 'root': root, 'e': e, 'w': w}

def reconstruct_tree(root_table: List[List[int]], i: int, j: int, terms: List[str]) -> Optional[Node]:
    if i > j:
        return None
    r = root_table[i][j]
    if r == 0:
        return None
    left = reconstruct_tree(root_table, i, r-1, terms)
    right = reconstruct_tree(root_table, r+1, j, terms)
    return Node(key=terms[r-1], index=r, left=left, right=right)

def tables_to_dataframes(e: List[List[float]], w: List[List[float]], root: List[List[int]]) -> Dict[str, pd.DataFrame]:
    n = len(root)-1
    e_df = pd.DataFrame([[e[i][j] for j in range(n+2)] for i in range(1,n+2)],
                        index=[f"i={i}" for i in range(1,n+2)],
                        columns=[f"j={j}" for j in range(0,n+2)])
    w_df = pd.DataFrame([[w[i][j] for j in range(n+2)] for i in range(1,n+2)],
                        index=[f"i={i}" for i in range(1,n+2)],
                        columns=[f"j={j}" for j in range(0,n+2)])
    root_df = pd.DataFrame([[root[i][j] for j in range(1,n+1)] for i in range(1,n+1)],
                           index=[f"i={i}" for i in range(1,n+1)],
                           columns=[f"j={j}" for j in range(1,n+1)])
    return {'e': e_df, 'w': w_df, 'root': root_df}

def tree_to_networkx(root_node: Node) -> nx.DiGraph:
    G = nx.DiGraph()
    def _visit(node: Optional[Node]):
        if node is None: return
        label = f"{node.key}\\n(idx={node.index})"
        G.add_node(node.index, label=label)
        if node.left:
            G.add_edge(node.index, node.left.index)
            _visit(node.left)
        if node.right:
            G.add_edge(node.index, node.right.index)
            _visit(node.right)
    _visit(root_node)
    return G

def plot_tree_networkx(root_node: Node, figsize=(8,6)) -> plt.Figure:
    G = tree_to_networkx(root_node)
    pos = hierarchy_pos(G, list(G.nodes)[0]) if len(G) > 0 else {}
    labels = nx.get_node_attributes(G, 'label')
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(1,1,1)
    nx.draw(G, pos=pos, labels=labels, with_labels=True, node_size=2000, font_size=8, arrows=False, ax=ax)
    ax.set_axis_off()
    plt.tight_layout()
    return fig

def hierarchy_pos(G, root=None, width=1.0, vert_gap=0.2, vert_loc=0, xcenter=0.5):
    # función auxiliar para posiciones tipo jerarquía
    if root is None:
        root = list(G.nodes)[0]
    def _hierarchy_pos(G, root, left, right, vert_loc, pos):
        pos[root] = ((left+right)/2.0, vert_loc)
        children = list(G.successors(root))
        if len(children)==0:
            return
        step = (right-left)/len(children)
        for i,ch in enumerate(children):
            _hierarchy_pos(G, ch, left + i*step, left + (i+1)*step, vert_loc-vert_gap, pos)
    pos = {}
    _hierarchy_pos(G, root, 0, width, vert_loc, pos)
    return pos

PY

In [ ]:
%%bash
cat > /content/streamlit_app.py <<'PY'

# streamlit_app.py
import streamlit as st
import pandas as pd
import tracemalloc
import time
import math
import random
from optimal_bst import optimal_bst, reconstruct_tree, tables_to_dataframes, plot_tree_networkx
import matplotlib.pyplot as plt
import io
import sys

st.set_page_config(page_title="Optimal BST - Demo + Benchmark", layout="wide")

st.title("Árbol de Búsqueda Binaria Óptimo (OBST) — Demo y Benchmark")
st.markdown("""
Calcula OBST (CLRS) / Knuth y permite ejecutar benchmarks empíricos.
Usa el panel `Entradas` para ejecutar un caso único y el panel `Benchmark` para comparar escalado.
""")

# ---------- Utilities ----------
def deep_getsizeof(obj, seen=None):
    """Estimación recursiva del tamaño en bytes de una estructura compuesta."""
    import sys
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    seen.add(obj_id)
    size = sys.getsizeof(obj)
    if isinstance(obj, dict):
        for k, v in obj.items():
            size += deep_getsizeof(k, seen)
            size += deep_getsizeof(v, seen)
    elif isinstance(obj, (list, tuple, set)):
        for item in obj:
            size += deep_getsizeof(item, seen)
    return size

def generate_terms(n: int):
    return [f"t{str(i).zfill(5)}" for i in range(1, n+1)]

def generate_probabilities(n: int, seed: int=None):
    rnd = random.Random(seed)
    p_raw = [rnd.random() for _ in range(n)]
    q_raw = [rnd.random() for _ in range(n+1)]
    total = sum(p_raw) + sum(q_raw)
    if total == 0:
        # fallback
        p = [1.0/(2*n) for _ in range(n)]
        q = [1.0/(2*(n+1)) for _ in range(n+1)]
    else:
        p = [v/total for v in p_raw]
        q = [v/total for v in q_raw]
    return p, q

def measure_one_run(n:int, method:str, seed:int=None):
    """Ejecuta optimal_bst para un input generado y mide tiempo, memoria y tamaño de tablas."""
    terms = generate_terms(n)
    p, q = generate_probabilities(n, seed=seed)
    tracemalloc.start()
    t0 = time.perf_counter()
    res = optimal_bst(terms, p, q, method=method)
    t1 = time.perf_counter()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    tables_mem = deep_getsizeof(res['e']) + deep_getsizeof(res['w']) + deep_getsizeof(res['root'])
    return {
        'n': n,
        'method': method,
        'time_s': (t1 - t0),
        'peak_mem_bytes': peak,
        'tables_mem_bytes': tables_mem,
        'cost': res['cost']
    }

# ---------- Left column: Single-run / interactive ----------
left, right = st.columns((1,1))

with left:
    st.header("Entradas / Ejecución individual")
    st.markdown("Ingresa términos + p + q (o usa los valores por defecto). Pulsa **Generar** para ver tablas y árbol.")
    method = st.selectbox("Método (ejecución única)", options=["knuth","clrs"], index=0)
    sort_terms = st.checkbox("Ordenar lexicográficamente y reordenar p", value=True)
    terms_txt = st.text_area("Términos (comma-separated)", value="assert,class,def,import", height=80)
    p_txt = st.text_area("p (probabilidades exitosas, comma-separated)", value="0.22,0.18,0.20,0.15", height=60)
    q_txt = st.text_area("q (probabilidades fallidas, comma-separated)", value="0.02,0.03,0.03,0.05,0.12", height=60)
    if st.button("Generar"):
        try:
            terms = [t.strip() for t in terms_txt.split(",") if t.strip()!='']
            p = [float(x.strip()) for x in p_txt.split(",") if x.strip()!='']
            q = [float(x.strip()) for x in q_txt.split(",") if x.strip()!='']
            if sort_terms:
                pairs = list(zip(terms, p))
                pairs.sort(key=lambda x: x[0])
                terms = [t for t,_ in pairs]
                p = [pi for _,pi in pairs]
                st.info("Términos ordenados lexicográficamente y p reordenado en el mismo orden.")
            st.write(f"n = {len(terms)}  — método: {method}")
            tracemalloc.start()
            t0 = time.perf_counter()
            res = optimal_bst(terms, p, q, method=method)
            t1 = time.perf_counter()
            current, peak = tracemalloc.get_traced_memory()
            tracemalloc.stop()
            cost = res['cost']
            st.metric("Costo esperado mínimo", f"{cost:.6f}")
            dfs = tables_to_dataframes(res['e'], res['w'], res['root'])
            st.subheader("Tabla e (costos)")
            st.dataframe(dfs['e'])
            st.subheader("Tabla w (probabilidades acumuladas)")
            st.dataframe(dfs['w'])
            st.subheader("Tabla root (raíces)")
            st.dataframe(dfs['root'])
            root_node = reconstruct_tree(res['root'], 1, len(terms), terms)
            if root_node:
                st.subheader("Árbol óptimo (visualización)")
                fig = plot_tree_networkx(root_node, figsize=(10,6))
                st.pyplot(fig)
            else:
                st.write("No se pudo reconstruir el árbol (root vacío).")
            # medidas
            st.subheader("Medidas empíricas (caso único)")
            st.write(f"Tiempo de ejecución: {(t1-t0):.6f} s")
            st.write(f"Memoria pico (tracemalloc): {peak/1024:.2f} KiB")
            tables_mem = deep_getsizeof(res['e']) + deep_getsizeof(res['w']) + deep_getsizeof(res['root'])
            st.write(f"Tamaño estimado de tablas (bytes): {tables_mem}")
        except Exception as exc:
            st.error(f"Error en la entrada o ejecución: {exc}")

# ---------- Right column: Benchmark ----------
with right:
    st.header("Benchmarking (comparativo)")
    st.markdown("""
    Ejecuta varias instancias para comparar escalado de los métodos.
    **Precaución:** elegir tamaños grandes y muchas repeticiones puede consumir tiempo y memoria en Colab.
    """)
    sizes_txt = st.text_input("Tamaños n (comma-separated)", value="10,30,60,100")
    methods_chk = st.multiselect("Métodos a comparar", options=["knuth","clrs"], default=["knuth","clrs"])
    repeats = st.number_input("Repeticiones por (n,method)", min_value=1, max_value=20, value=3, step=1)
    random_seed = st.number_input("Seed para generación aleatoria (0 para variable)", value=0, step=1)
    run_bench = st.button("Run benchmark")
    if run_bench:
        try:
            sizes = [int(x.strip()) for x in sizes_txt.split(",") if x.strip()!='']
            if len(methods_chk) == 0:
                st.error("Selecciona al menos un método.")
            else:
                # prepare results
                results = []
                progress = st.progress(0)
                total_tasks = len(sizes) * len(methods_chk) * repeats
                task_count = 0
                with st.spinner("Running benchmarks... esto puede tardar varios segundos/minutos según parámetros"):
                    for n in sizes:
                        for method_b in methods_chk:
                            for rep in range(repeats):
                                seed = (random_seed if random_seed != 0 else None)
                                row = measure_one_run(n, method_b, seed=seed)
                                results.append(row)
                                task_count += 1
                                progress.progress(task_count / total_tasks)
                df = pd.DataFrame(results)
                st.success("Benchmark completado.")
                st.subheader("Tabla de resultados")
                st.dataframe(df.sort_values(['n','method']).reset_index(drop=True))
                # Plots
                st.subheader("Gráficos")
                # 1) time vs n (log-log)
                fig1, ax1 = plt.subplots()
                for m, g in df.groupby('method'):
                    g2 = g.groupby('n')['time_s'].mean().reset_index()
                    ax1.plot(g2['n'], g2['time_s'], marker='o', label=m)
                ax1.set_xlabel("n")
                ax1.set_ylabel("time (s)")
                ax1.set_xscale('linear')
                ax1.set_yscale('log')
                ax1.set_title("Tiempo medio vs n (escala log en Y)")
                ax1.legend()
                st.pyplot(fig1)
                # 2) memory vs n (peak_mem_bytes mean)
                fig2, ax2 = plt.subplots()
                for m, g in df.groupby('method'):
                    g2 = g.groupby('n')['peak_mem_bytes'].mean().reset_index()
                    ax2.plot(g2['n'], g2['peak_mem_bytes']/1024.0, marker='o', label=m)  # KiB
                ax2.set_xlabel("n")
                ax2.set_ylabel("peak mem (KiB)")
                ax2.set_title("Memoria pico media vs n")
                ax2.legend()
                st.pyplot(fig2)
                # 3) normalized curves: time/n^2 and time/n^3 (mean)
                fig3, ax3 = plt.subplots(1,2, figsize=(12,4))
                for m, g in df.groupby('method'):
                    g2 = g.groupby('n')['time_s'].mean().reset_index()
                    ax3[0].plot(g2['n'], g2['time_s'] / (g2['n']**2), marker='o', label=m)
                    ax3[1].plot(g2['n'], g2['time_s'] / (g2['n']**3), marker='o', label=m)
                ax3[0].set_xlabel("n"); ax3[0].set_ylabel("time / n^2"); ax3[0].set_title("time / n^2")
                ax3[1].set_xlabel("n"); ax3[1].set_ylabel("time / n^3"); ax3[1].set_title("time / n^3")
                ax3[0].legend(); ax3[1].legend()
                st.pyplot(fig3)
                # Download CSV
                csv_buf = df.to_csv(index=False).encode('utf-8')
                st.download_button("Download results CSV", data=csv_buf, file_name="obst_benchmark_results.csv", mime="text/csv")
                # basic summary text
                st.markdown("**Resumen rápido:** la gráfica `time / n^2` ayuda a evaluar si la complejidad efectiva se acerca a O(n^2) (curva estable) o se aleja hacia O(n^3).")
        except Exception as e:
            st.error(f"Error durante benchmark: {e}")

# ---------- Footer ----------
st.markdown("---")
st.markdown("Nota: en Colab el tiempo y memoria pueden variar por recursos temporales y limitaciones de la VM. Para benchmarking más estable, ejecuta localmente o en una VM dedicada.")

PY

In [ ]:
from getpass import getpass
authtoken = getpass("Introduce tu ngrok authtoken (no se mostrará): ")

from pyngrok import ngrok

ngrok.set_auth_token(authtoken)
print("Authtoken configurado correctamente.")

Introduce tu ngrok authtoken (no se mostrará): ··········
Authtoken configurado correctamente.


In [ ]:
# Ejecuta Streamlit en background y crea un túnel ngrok público
import nest_asyncio, subprocess, time, os
from pyngrok import ngrok

nest_asyncio.apply()

# abre el túnel en el puerto 8501 y obtiene la URL pública
public_tunnel = ngrok.connect(8501, bind_tls=True)
print("URL pública (ngrok):", public_tunnel.public_url)

# lanzar streamlit en background
cmd = "streamlit run /content/streamlit_app.py --server.port 8501 --server.headless true"
proc = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)

time.sleep(3)
print("Streamlit lanzado en background (PID:", proc.pid, "). Revisa la URL pública arriba.")

URL pública (ngrok): https://b630ec55aae0.ngrok-free.app
Streamlit lanzado en background (PID: 397 ). Revisa la URL pública arriba.
